In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [51]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [52]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
class_name = 'chair'

do_jsd = False
do_mmd = False
do_cov = True

mmd_loss = 'emd'

skip = ['train', 'test']

boost_samples = 1
n_reps = 3
subsample = True # new - for rebuttal to get entire dataset

In [61]:
if do_jsd:
    boost_samples = 1
else:
    boost_samples = boost_samples
assert(np.sum([do_cov, do_jsd, do_jsd]) == 1)

In [62]:
train_data = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/chair_train.npz'
sample_files = [train_data]

In [63]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [64]:
if do_mmd:
    func_ = evaluator.compute_mmd    
elif do_cov:
    func_ = evaluator.compute_coverage    

In [65]:
trials_res = []
for _ in range(n_reps):
    for sample_file in sample_files:
        evaluator.prepare_sample_data(sample_file, boost_samples)
        
        if not subsample: # new for rebuttal
            evaluator.sample_data['test'] = evaluator.sample_data['train']
            evaluator.sample_data['val'] = evaluator.sample_data['train']
        
        sample_name = osp.basename(sample_file)[:-len('.npz')]
        
        print sample_name

        if do_jsd:
            res = evaluator.compute_jsd()
        else:            
            res = func_(loss=mmd_loss, sample_estimator=False, skip=skip)
            
            trials_res.append(res)    

chair_train
val 0.523668639053 0.0
chair_train
val 0.5 0.0
chair_train
val 0.53550295858 0.0


In [66]:
split = 'val'
avgs = []
for i in range(n_reps):
    avgs.append(np.mean(trials_res[i][split]))
print avgs
print np.mean(avgs)

[0.52366863905325445, 0.5, 0.53550295857988162]
0.519723865878
